In [1]:
import os
import pandas as pd
import cx_Oracle
from dotenv import load_dotenv

### Inicializálás

In [2]:
load_dotenv()
username = os.getenv('ORACLE_USERNAME')
password = os.getenv('ORACLE_PASSWORD')
dsn = os.getenv('ORACLE_DSN')
pool = cx_Oracle.SessionPool(user=username, password=password, dsn=dsn, min=2, max=5, increment=1)

input_file = 'BETÖLTŐ_reki_06.xlsx'
output_file = 'elokeszit.xlsx'
KID='357328/2024'
AHT='357328'

def get_data(ksh):
    query = f'''
select 
    onk.ksh, 
    onk.onev, 
    SUBSTR(st.torzs_sz||'-'||st.afakod||'-'||st.mk, 0, 8) as torzsszam, 
    cim.irszam, cim.koztnev||' '||cim.kozjell as cim, 
    cim.hsz1 as hsz, 
    TRIM(rk.megnevezes) as regio
from SUPPORTSYS.t_onkorm_tech onk
left join (select * from SUPPORTSYS.t_stat) st on st.ksh=onk.ksh
left join (select * from SUPPORTSYS.t_cim) cim on cim.ksh=onk.ksh
left join SUPPORTSYS.t_regio rk on rk.regiokod=onk.rk
where onk.ksh='{ksh}'
'''
    connection = pool.acquire()
    cursor = connection.cursor()
    cursor.execute(query)
    data = cursor.fetchall()    
    pool.release(connection)
    return data

### EBR betöltő beolvasás

In [3]:
df = pd.read_excel(f'./data/{input_file}', usecols=[1, 2, 6], names=['KSH', 'ONK', 'AMOUNT'])
df['KSH'] = df['KSH'].astype(str).str.zfill(7)
df['KID'] = KID
df['AHT'] = AHT
df['ONEV'] = ''
df['TORZSSZAM'] = ''
df['IRSZAM'] = ''
df['CIM'] = ''
df['HSZ'] = ''
df['REGIO'] = ''

df.head()

,KSH,ONK,AMOUNT,KID,AHT,ONEV,TORZSSZAM,IRSZAM,CIM,HSZ,REGIO
0,1612441,Abádszalók,8801115,357328/2024,357328,,,,,,
1,0526718,Abaújkér,3804205,357328/2024,357328,,,,,,
2,0510357,Abod,129238,357328/2024,357328,,,,,,
3,1118139,Ácsteszér,1998808,357328/2024,357328,,,,,,
4,1906673,Ajka,44757839,357328/2024,357328,,,,,,


### Szükséges adatok az adatbázisból

In [4]:
column_names = ['ONEV', 'TORZSSZAM', 'IRSZAM', 'CIM', 'HSZ', 'REGIO']

for index, row in df.iterrows():
    ksh_value = row['KSH']
    data = get_data(ksh_value)
    for i, column_name in enumerate(column_names):
        df.at[index, column_name] = data[0][i+1]
df.head()

,KSH,ONK,AMOUNT,KID,AHT,ONEV,TORZSSZAM,IRSZAM,CIM,HSZ,REGIO
0,1612441,Abádszalók,8801115,357328/2024,357328,Abádszalók,15733074,5241,Deák Ferenc Út,12,Észak-Alföld
1,0526718,Abaújkér,3804205,357328/2024,357328,Abaújkér,15545064,3882,Rákóczi Utca,26,Észak-Magyarország
2,0510357,Abod,129238,357328/2024,357328,Abod,15548122,3753,Magyar Út,42,Észak-Magyarország
3,1118139,Ácsteszér,1998808,357328/2024,357328,Ácsteszér,15385437,2887,Kossuth Lajos Utca,45,Közép-Dunántúl
4,1906673,Ajka,44757839,357328/2024,357328,Ajka,15734185,8400,Szabadság Tér,12,Közép-Dunántúl


### Bezárás

In [5]:
pool.close()
df.to_excel(f'./data/{output_file}', index=False)